# Annotation conversions

Notebook for performing a series of manipulations to the genome annotation files. Poorly documented.

2020-09-14

## Initial boilerplate

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = ''
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))

print('Standard variables loaded, you are good to go!')

Python-dotenv could not parse statement starting at line 1


Standard variables loaded, you are good to go!


## 1. Annotation file

Some lines did not have gene id, which made DE tools crash. Therefore we add them here as the transcript id:

In [2]:
old_genome_name = join(RAW_EXTERNAL,'genome_annotation.gtf')
fixed_genome_name = join(RAW_EXTERNAL,'genome_annotation_fixed.gtf')
with open(old_genome_name) as old_genome:
    with open(fixed_genome_name, 'w', newline='') as fixed_genome:
        lines = []
        for line in old_genome:
            if "gene_id" not in line:
                line = line.replace("transcript_id", "gene_id")
            line = line.replace("; ",";")
            lines.append(line)
        fixed_genome.writelines(lines)

## 2. Gene length file

We calculated the gene length by substracting the end from start of each exon, and then adding up all exons.

In [3]:
import csv
genome_file_name = join(RAW_EXTERNAL,'genome_annotation_fixed.gtf')
length_file_name = join(RAW_EXTERNAL,'gene_length.tabular')

# create list of gene names:
with open(genome_file_name) as genome_file:
    gene_names = []
    for line in csv.reader(genome_file, delimiter='\t'):
        if line[2] == "exon" and "gene_name" in line[8]:
            gene_name = line[8].split(";")[2]
            gene_name = gene_name.replace('gene_name ','')
            gene_name = gene_name.replace('"','')
            if gene_name not in gene_names:
                gene_names.append(gene_name)

# compute length for each gene:
with open(length_file_name, 'w', newline='') as length_file:
    length_writer = csv.writer(length_file, delimiter='\t')
    for gene_name in gene_names:
        gene_length = 0
        with open(genome_file_name) as genome_file:
            for line in csv.reader(genome_file, delimiter='\t'):
                if line[2] == "exon" and f'gene_name "{gene_name}"' in line[8]:
                    # add length of exon
                    gene_length += int(line[4])-int(line[3])+1
            # write new gene:
            length_writer.writerow([gene_name,gene_length])

## 3. Ontology file

Created file with associations 1-1 of each gene with ontologies.

In [4]:
old_ontology_name = join(RAW_EXTERNAL,'ontologies.tabular')
new_ontology_name = join(RAW_EXTERNAL,'ontologies_fixed.tabular')

with open(old_ontology_name) as old_ontology:
    with open(new_ontology_name, 'w', newline='') as new_ontology:
        old_reader = csv.reader(old_ontology, delimiter='\t')
        new_writer = csv.writer(new_ontology, delimiter='\t')
        for line in old_reader:
            if "|" in line[2]:
                GO_list = line[2].split("|")
                for GO in GO_list:
                    new_writer.writerow([line[1], GO])
            else:
                new_writer.writerow([line[1], line[2]])

Now do it again but for phophoproteomics data:

In [5]:
# Load list of phosphoproteomic ids:
import pandas as pd
phospho_file = join(INTERMEDIATE,'phosphoCounts_normalized.csv')
phospho_ids = pd.read_csv(phospho_file, index_col=0, sep=",").index.tolist()

# Load protein ids as dictionary:
prot_ids = {}
with open(join(RAW_EXTERNAL,'uniprot_genes.tab')) as f:
    for line in f:
        try:
            (prot, gene) = line.split()
            prot_ids[gene] = prot
        except:
            pass

# Create phophoproteomics ontology file:        
old_ontology_name = join(RAW_EXTERNAL,'ontologies.tabular')
new_ontology_name = join(RAW_EXTERNAL,'ontologies_phospho.tabular')

with open(old_ontology_name) as old_ontology:
    with open(new_ontology_name, 'w', newline='') as new_ontology:
        old_reader = csv.reader(old_ontology, delimiter='\t')
        new_writer = csv.writer(new_ontology, delimiter='\t')
        for line in old_reader:
            try:
                # Match to the desired protein:
                protein = prot_ids[line[1]]
                phospho_matches = [match for match in phospho_ids if f"{protein}_" in match]
                for phospho_id in phospho_matches:
                    # Add all necessary GO term lines:
                    if "|" in line[2]:
                        GO_list = line[2].split("|")
                        for GO in GO_list:
                            new_writer.writerow([phospho_id, GO])
                    else:
                        new_writer.writerow([phospho_id, line[2]])
            except KeyError:
                pass